We will consider forward substitution and an iteration scheme for linear systems.

# Exercise 1: Forward substitution

Let $L$ be an $n\times n$ lower triangular matrix with non-zero diagonal entries. We want to solve

$$ Lx = b.$$

More explicitly:

$$\left[\begin{array}{cccccccc} l_{11} & 0 & 0 &0 & \cdots & 0 \\ l_{21} & l_{22} & 0 & 0 & \cdots & 0 \\ l_{31} & l_{32} & l_{33} & 0 & \cdots & 0 \\ \vdots & &&&& \vdots \\ l_{n1} & l_{n2} & \cdots && \cdots & l_{nn}\end{array}\right] \left[ \begin{array}{c} x_1 \\ x_2 \\ \vdots \\ \vdots \\ x_n \end{array} \right] = \left[ \begin{array}{c} b_1 \\ b_2 \\ \vdots \\ \vdots \\ b_n \end{array} \right].$$

This gives the set of equations

$$ \begin{array}{l} x_1 = b_1/l_{11}\\ x_2 = (b_2 - l_{21} x_1)/l_{22}\\ x_3 = (b_3 - l_{31} x_1 - l_{32} x_2)/l_{33}\\ \vdots \end{array} $$

And in general

$$ x_i = \left( b_i - \sum_{j=1}^{i-1} l_{ij} x_j \right)/l_{ii}. $$

### Question
Write a function that implements the above algorithm.

### Solution

In [1]:
import numpy as np 

def forward_substitution(L, b):
    x = np.zeros_like(b)
    for i in range(len(b)):
        SUM = 0.0
        for j in range(i):
            SUM += L[i,j]*x[j]
        x[i] = (b[i] - SUM)/L[i,i]
    return x

### Question 

Test your algorithm on the following matrices: 

In [2]:
n = 10
L = np.random.rand(n,n)
b = np.random.rand(n,1)

for i in range(n):
    for j in range(i+1,n):
        L[i,j] = 0.0
print L

[[ 0.40915467  0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.10130172  0.48043902  0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.46789344  0.94116663  0.38074796  0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.29563007  0.30944301  0.282063    0.01313491  0.          0.          0.
   0.          0.          0.        ]
 [ 0.6604978   0.63075808  0.27418114  0.70569672  0.69898751  0.          0.
   0.          0.          0.        ]
 [ 0.61892999  0.25087098  0.23478928  0.49790787  0.68931452  0.87458669
   0.          0.          0.          0.        ]
 [ 0.58968844  0.89777457  0.66088146  0.69225966  0.26067606  0.58300746
   0.74994329  0.          0.          0.        ]
 [ 0.39780998  0.55716451  0.47033898  0.79959631  0.93517522  0.0175314
   0.49384161  0.94624836  0.          0.        ]
 [ 0.94145894  0.30140266  0.758592    0.

### Solution

In [3]:
L_inverse = np.linalg.inv(L)
x = np.dot(L_inverse, b)
x - forward_substitution(L, b)

array([[  7.21644966e-15],
       [ -1.11022302e-15],
       [ -7.54951657e-15],
       [ -1.42108547e-14],
       [ -1.42108547e-14],
       [  1.06581410e-14],
       [ -3.55271368e-15],
       [  2.48689958e-14],
       [ -1.06581410e-14],
       [  2.48689958e-14]])

# Exercise 2: An iterative approach

Create a matrix $A$:

In [4]:
n = 4
shape = (n,n)
A = np.zeros(shape)
for i in range(n):
    for j in range(n):
        dif = i-j
        if i >= j: 
            A[i,j] = (-1)**dif*(dif+1)
        else:
            A[i,j] = (0.1)**(-dif)
print A

[[  1.00000000e+00   1.00000000e-01   1.00000000e-02   1.00000000e-03]
 [ -2.00000000e+00   1.00000000e+00   1.00000000e-01   1.00000000e-02]
 [  3.00000000e+00  -2.00000000e+00   1.00000000e+00   1.00000000e-01]
 [ -4.00000000e+00   3.00000000e+00  -2.00000000e+00   1.00000000e+00]]


Let $U$ be a matrix that coincides with $A$ above the diagonal, but is zero on the diagonal and below:


In [5]:
U = A.copy() # not U = A
for i in range(n):
    for j in range(n):
        if i >= j: 
            U[i,j] = 0.0
print U 

[[ 0.     0.1    0.01   0.001]
 [ 0.     0.     0.1    0.01 ]
 [ 0.     0.     0.     0.1  ]
 [ 0.     0.     0.     0.   ]]


Similarly, let $L$ be a matrix that differs from $A$ only above the diagonal, where $L$ is zero:  

In [6]:
L = A.copy() # not L = A
for i in range(n):
    for j in range(n):
        if i < j: 
            L[i,j] = 0.0
print L 

[[ 1.  0.  0.  0.]
 [-2.  1.  0.  0.]
 [ 3. -2.  1.  0.]
 [-4.  3. -2.  1.]]


Then solving $Ax = b$ is equivalent to solving 

\begin{equation} 
Lx = b - Ux.
\end{equation} 

Though the LHS is of the form of an equation amenable to forward substitution, the RHS is not: it depends on $x$ (instead of being constant). The key point is that the solution of $Lx = b - Ux$ can be viewed as the fixed point of the function 

\begin{equation} 
g(x) = L^{-1} (b - Ux).
\end{equation} 



### Question 

Write down on a sheet of paper how the fixed-point method can be combined with back-substitution to solve for $x$. 



### Solution 

Consider the fixed-point method: 

\begin{eqnarray} 
x_{n+1} = g(x_n). 
\end{eqnarray}

Multiplying across by $L^{-1}$, we get: 

\begin{eqnarray} 
Lx_{n+1} = b - U x_n,
\end{eqnarray}

i.e. the next term in the fixed-point sequence is obtained from the previous term by solving the system, 

\begin{eqnarray} 
Lx_{n+1} = b_n,
\end{eqnarray}

where $b_n = b - U x_n$. This we can do by forward substitution. 


### Question 

Implement your iterative algorithm in Python for a vector $b$ consisting entirely of ones and a starting $x$-value of all zeros. Test the solution so obtained.

### Solution

In [7]:
shape = (n,1)
b = np.ones(shape)
x = np.zeros_like(b)
for i in range(30):
    x = forward_substitution(L, b - np.dot(U,x))
print 'Ax - b:'
print np.dot(A, x) - b
print 

Ax - b:
[[ -8.27116153e-14]
 [ -1.00031095e-13]
 [ -6.03961325e-14]
 [  0.00000000e+00]]

